<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> TARANTELLI Alix & Chloe BROCHET - LOSTIE de KERHOR - Group 12
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [2]:
tf.reset_default_graph()

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288662004
Epoch:  02   =====> Loss= 0.732692676
Epoch:  03   =====> Loss= 0.599985538
Epoch:  04   =====> Loss= 0.536749699
Epoch:  05   =====> Loss= 0.497753732
Epoch:  06   =====> Loss= 0.471234547
Epoch:  07   =====> Loss= 0.451261122
Epoch:  08   =====> Loss= 0.435740518
Epoch:  09   =====> Loss= 0.423274494
Epoch:  10   =====> Loss= 0.413069920
Epoch:  11   =====> Loss= 0.404554589
Epoch:  12   =====> Loss= 0.396918491
Epoch:  13   =====> Loss= 0.390203175
Epoch:  14   =====> Loss= 0.384384598
Epoch:  15   =====> Loss= 0.379245824
Epoch:  16   =====> Loss= 0.374483929
Epoch:  17   =====> Loss= 0.370222854
Epoch:  18   =====> Loss= 0.366388978
Epoch:  19   =====> Loss= 0.363033491
Epoch:  20   =====> Loss= 0.359776941
Epoch:  21   =====> Loss= 0.356680659
Epoch:  22   =====> Loss= 0.354078967
Epoch:  23   =====> Loss= 0.351193055
Epoch:  24   =====> Loss= 0.348619844
Epoch:  25   =====> Loss= 0.346619813
Epoch:  26   =====> Loss= 0.344197939
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


<div class="alert alert-success">
Here is how we proceeded to obtain a 99% accuracy : <br>
First, we tested the model with the **SDG optimizer** on 40 epochs. Then we decided to compare the performances of this optimizer to the **Adam's optimizer**. <br>
Second, we wanted to see the **impact of adding a dropout layer** in the model. We tested the model with both optimizers. <br>
Third, we decided to **increase the number of epochs** to a 100 to see how this parameter increases accuracy. To do so, we had to modifiy the Loss function when using the Adam's optimizer. <br>
**Finally, our best accuracy was achieved when using Adam's optimizer, no dropout layer and for 100 epochs.** 
</div>

In [3]:
# Functions for weigths and bias initilization 
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [4]:
def LeNet5_Model(data):    
    # your inmplementation goes here
    
    # ----- Input layer -----
    input_layer = tf.reshape(data, [-1,28, 28, 1]) 
    print(input_layer)
    
    # ----- Layer 1: Convolution & Activation-----
    filter_conv1 = weight_variable([5,5,1,6])
    strides_conv1 = [1,1,1,1]
    biais_conv1 = bias_variable([6])
    conv1 = tf.nn.relu(tf.nn.conv2d(input_layer, filter_conv1, strides = strides_conv1,  padding = "SAME") + biais_conv1 )
    print(conv1)
 
    # ----- Layer 1: MaxPooling -----
    strides_maxpool1 = [1,2,2,1]
    maxPooling1 = tf.nn.max_pool(conv1, ksize=[1,2, 2,1], strides=strides_maxpool1, padding = 'VALID')
    print(maxPooling1)
    
     # ----- Layer 2: Convolution & Activation-----
    filter_conv2 = weight_variable([5,5,6,16])
    strides_conv2 = [1,1,1,1]
    biais_conv2 = bias_variable([16])
    conv2 = tf.nn.relu(tf.nn.conv2d(maxPooling1, filter_conv2, strides = strides_conv2,  padding = "VALID") + biais_conv2 )
    print(conv2)
 
    # ----- Layer 2: MaxPooling -----
    strides_maxpool2 = [1,2,2,1]
    maxPooling2 = tf.nn.max_pool(conv2, ksize=[1,2, 2,1], strides=strides_maxpool2, padding = 'VALID')
    print(maxPooling2)
    
    # ----- Flatten -----
    flattened_output = tf.reshape(maxPooling2,[-1,5*5*16])
    print(flattened_output)
    
    # ----- Layer 3: Fully connected & activation -----
    weight_connection3 = weight_variable([400,120])
    biais_connection3 = bias_variable([120])
    connection3 = tf.nn.relu(tf.matmul(flattened_output,weight_connection3)+biais_connection3)
    print(connection3)
    
    # ----- Layer 4: Fully connected & activation -----
    weight_connection4 = weight_variable([120,84])
    biais_connection4 = bias_variable([84])
    connection4 = tf.nn.relu(tf.matmul(connection3,weight_connection4)+biais_connection4)
    print(connection4)
    
    # ----- Layer 5: Fully connected & activation -----
    weight_connection5 = weight_variable([84,10])
    biais_connection5 = bias_variable([10])
    connection5 = tf.nn.softmax(tf.matmul(connection4,weight_connection5)+biais_connection5)
    print(connection5)
    return (connection5)
 
    
LeNet5_Model(X_train) 
   

Tensor("Reshape:0", shape=(55000, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(55000, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(55000, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(55000, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(55000, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(55000, 400), dtype=float32)
Tensor("Relu_2:0", shape=(55000, 120), dtype=float32)
Tensor("Relu_3:0", shape=(55000, 84), dtype=float32)
Tensor("Softmax:0", shape=(55000, 10), dtype=float32)


<tf.Tensor 'Softmax:0' shape=(55000, 10) dtype=float32>

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

<div class="alert alert-success">
For the pooling layers, there is no parameters.

For the convolution layers: <br>
Weights = (K x L x D1) x F <br>
Biaises = F

__K__: filter height <br>
__L__: filter weight <br>
__D__: input depth <br>
__F__: output depth <br>


**Convolution Layer 1 :** <br>
K = 5 <br>
L= 5 <br>
D = 1 <br>
F = 6 <br>
So, wieghts = 150, biases = 6 <br>


**Convolution Layer 2 :** <br>
K = 5 <br>
L= 5 <br>
D = 6 <br>
F = 16 <br>
So, wieghts = 2 400, biases = 16
</div>

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Fucntion: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [5]:
tf.reset_default_graph()

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'SDG_files/'
saving_path ='models/'

# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='PreData')
model = LeNet5_Model(x)

# ----- Loss function -----
with tf.name_scope('Loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
   
# ----- Accuracy -----
acc = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
acc = tf.reduce_mean(tf.cast(acc, tf.float32))

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)


<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [6]:
def evaluate(logits, labels):
    # logits will be the outputs of your model, labels will be one-hot vectors corresponding to the actual labels
    # logits and labels are numpy arrays
    # this function should return the accuracy of your model
    acc = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    return(acc)

with tf.name_scope('Accuracy'):
    acc = evaluate(model,y)

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [7]:
from time import time 

In [8]:


# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss_SGD", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_SGD", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

display_step = 1

with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initializing the variables
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    # optimizer and cost are the same kinds of objects as in Section 1
    # Train your model
    # Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
    # Print the accuracy on testing data
    with tf.Session() as sess:
        #train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        saver = tf.train.Saver()
        t1 = time()
        print("Starting Training")
    
        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)
            # Loop over all batches
            for i in range(total_batch):
            #print(i)
                batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
                #batch_xs = batch_xs.reshape(-1, 784)
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost), "Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))
        print("Training Finished!")
        t2= time ()
        savemodel = saver.save(sess,saving_path)

        # Test model
        # Calculate accuracy
        print("Final Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}), "Training Time", t2-t1)
        summary_writer.flush()

In [10]:
train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Starting Training
Epoch:  01   =====> Loss= 2.312004849 Accuracy: 0.1267
Epoch:  02   =====> Loss= 2.300945116 Accuracy: 0.1447
Epoch:  03   =====> Loss= 2.293289301 Accuracy: 0.169
Epoch:  04   =====> Loss= 2.285715579 Accuracy: 0.1996
Epoch:  05   =====> Loss= 2.276157175 Accuracy: 0.2432
Epoch:  06   =====> Loss= 2.262128219 Accuracy: 0.3196
Epoch:  07   =====> Loss= 2.238467530 Accuracy: 0.4379
Epoch:  08   =====> Loss= 2.191733862 Accuracy: 0.515
Epoch:  09   =====> Loss= 2.086680934 Accuracy: 0.5651
Epoch:  10   =====> Loss= 1.821873232 Accuracy: 0.6817
Epoch:  11   =====> Loss= 1.291416404 Accuracy: 0.7817
Epoch:  12   =====> Loss= 0.830878836 Accuracy: 0.8318
Epoch:  13   =====> Loss= 0.617810701 Accuracy: 0.8576
Epoch:  14   =====> Loss= 0.514609978 Accuracy: 0.8751
Epoch:  15   =====> Loss= 0.453673803 Accuracy: 0.8848
Epoch:  16   =====> Loss= 0.412516177 Accuracy: 0.8944
Epoch:  17   =====> Loss= 0.382832161 Accuracy: 0.8997
Epoch:  18   =====> Loss= 0.359339810 Accuracy: 0

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.


<img src="MNIST_figures/SDG_Accuracy.JPG",width="800" height="600" align="center">
<center><span>Figure 2a: Accuracy </span></center>


<img src="MNIST_figures/SDG_Loss.JPG",width="800" height="600" align="center">
<center><span>Figure 2b: Loss </span></center>

<div class="alert alert-success">
When we use the Stochastic Gradient Descent as optimizer, with 40 epochs, we got an accuracy 0.9523 in 627s
It reaches an 80% accuracy really fastly then increases slowly. 
</div>

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |         0.9523     |  epoch 20: 0.989    |       
| Training Time        |        627,74      |      626,91         | 
| Training Time / epoch|         15,69      |       15,67         |


- Which optimizer gives the best accuracy on test data?
<div class="alert alert-success">
With the Adam optimizer, we got a higher accuracy in fewer epochs. We faced a problem of convergence starting from the 20th epoch  because of the Loss function. Indeed, the loss is so small that taking its log is like computing log(0). To face this issue, we should add a small constant in the log function. <br>
We notice that the accuracy is very high for the first epoch (0.97). Then it slowly increases over the next epochs. It seems that running over 40 epochs is too high and that it may be a lost of time.
</div>

In [11]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'Adam_files/'
saving_path ='models/'
display_step = 1
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model(x)

# ----- Loss function -----
with tf.name_scope('LossAdam'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
    
with tf.name_scope('AccuracyAdam'):
    acc = evaluate(model,y)


with tf.name_scope('Adam'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_Adam", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_Adam", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  01   =====> Loss= 0.361617278 Accuracy: 0.9701
Epoch:  02   =====> Loss= 0.090087794 Accuracy: 0.9782
Epoch:  03   =====> Loss= 0.062645129 Accuracy: 0.9848
Epoch:  04   =====> Loss= 0.049700349 Accuracy: 0.9858
Epoch:  05   =====> Loss= 0.041520815 Accuracy: 0.9876
Epoch:  06   =====> Loss= 0.034721661 Accuracy: 0.9881
Epoch:  07   =====> Loss= 0.029558253 Accuracy: 0.9884
Epoch:  08   =====> Loss= 0.025184028 Accuracy: 0.9893
Epoch:  09   =====> Loss= 0.020145406 Accuracy: 0.989
Epo

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [12]:
def LeNet5_Model_Dropout(image):    
    # your implementation goes here
    # ----- Input layer -----
    input_layer = tf.reshape(image, [-1,28, 28, 1]) 
    print(input_layer)
    
    # ----- Layer 1: Convolution & Activation-----
    filter_conv1 = weight_variable([5,5,1,6])
    strides_conv1 = [1,1,1,1]
    biais_conv1 = bias_variable([6])
    conv1 = tf.nn.relu(tf.nn.conv2d(input_layer, filter_conv1, strides = strides_conv1,  padding = "SAME") + biais_conv1 )
    print(conv1)
 
    # ----- Layer 1: MaxPooling -----
    strides_maxpool1 = [1,2,2,1]
    maxPooling1 = tf.nn.max_pool(conv1, ksize=[1,2, 2,1], strides=strides_maxpool1, padding = 'VALID')
    print(maxPooling1)
    
     # ----- Layer 2: Convolution & Activation-----
    filter_conv2 = weight_variable([5,5,6,16])
    strides_conv2 = [1,1,1,1]
    biais_conv2 = bias_variable([16])
    conv2 = tf.nn.relu(tf.nn.conv2d(maxPooling1, filter_conv2, strides = strides_conv2,  padding = "VALID") + biais_conv2 )
    print(conv2)
 
    # ----- Layer 2: MaxPooling -----
    strides_maxpool2 = [1,2,2,1]
    maxPooling2 = tf.nn.max_pool(conv2, ksize=[1,2, 2,1], strides=strides_maxpool2, padding = 'VALID')
    print(maxPooling2)
    
    # ----- Flatten -----
    flattened_output = tf.reshape(maxPooling2,[-1,5*5*16])
    print(flattened_output)
    
    #adding dropout 
    droplayer = tf.nn.dropout(flattened_output, 0.75)
    print(droplayer)
    
    # ----- Layer 3: Fully connected & activation -----
    weight_connection3 = weight_variable([400,120])
    biais_connection3 = bias_variable([120])
    connection3 = tf.nn.relu(tf.matmul(droplayer,weight_connection3)+biais_connection3)
    print(connection3)
    
    # ----- Layer 4: Fully connected & activation -----
    weight_connection4 = weight_variable([120,84])
    biais_connection4 = bias_variable([84])
    connection4 = tf.nn.relu(tf.matmul(connection3,weight_connection4)+biais_connection4)
    print(connection4)
    
    # ----- Layer 5: Fully connected & activation -----
    weight_connection5 = weight_variable([84,10])
    biais_connection5 = bias_variable([10])
    connection5 = tf.nn.softmax(tf.matmul(connection4,weight_connection5)+biais_connection5)
    print(connection5)
    return (connection5)

LeNet5_Model_Dropout(X_train) 
 

Tensor("Reshape:0", shape=(55000, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(55000, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(55000, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(55000, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(55000, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(55000, 400), dtype=float32)
Tensor("dropout/mul:0", shape=(55000, 400), dtype=float32)
Tensor("Relu_2:0", shape=(55000, 120), dtype=float32)
Tensor("Relu_3:0", shape=(55000, 84), dtype=float32)
Tensor("Softmax:0", shape=(55000, 10), dtype=float32)


<tf.Tensor 'Softmax:0' shape=(55000, 10) dtype=float32>

In [26]:
tf.reset_default_graph()
learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'drop_files/'
saving_path ='models/'
display_step = 1
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model_Dropout(x)

# ----- Loss function -----
with tf.name_scope('Lossdrop'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
    
with tf.name_scope('Accuracydrop'):
    acc = evaluate(model,y)

with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_drop", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_drop", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  01   =====> Loss= 2.290595883 Accuracy: 0.1091
Epoch:  02   =====> Loss= 2.262853019 Accuracy: 0.1642
Epoch:  03   =====> Loss= 2.225646502 Accuracy: 0.2685
Epoch:  04   =====> Loss= 2.156752956 Accuracy: 0.3817
Epoch:  05   =====> Loss= 2.009450127 Accuracy: 0.4766
Epoch:  06   =====> Loss= 1.723772571 Accuracy: 0.5599
Epoch:  07   =====> Loss= 1.366263479 Accuracy: 0.6353
Epoch:  08   =====> Loss= 1.101870189 Accuracy: 0.6891
Ep

<div class="alert alert-success">
We first chose to try the model with the dropout with the SDG optimizer to fully see its impact. 
The purpose of a dropout layer is to fix the overfitting problem. 
As some nodes are droped it is also supposed to be faster. It is 7 seconds faster. 
Yet, the final accurancy is lower than without using the dropout layer (0.93 vs 0.95)
That may be because our training set is really similar and overfitting is the solution
With this training set, using the model without a dropout is more efficient. 
<\div>

In [28]:
tf.reset_default_graph()
learning_rate = 0.001
training_epochs = 20
batch_size = 128
logs_path = 'Adam_drop_files/'
saving_path ='models/'
display_step = 1
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model_Dropout(x)

# ----- Loss function -----
with tf.name_scope('Loss_drop_adam'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
    
with tf.name_scope('Accuracy_drop_adam'):
    acc = evaluate(model,y)

with tf.name_scope('Adamdrop'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
        
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_drop", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_drop", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  01   =====> Loss= 0.392393920 Accuracy: 0.9629
Epoch:  02   =====> Loss= 0.113279319 Accuracy: 0.9748
Epoch:  03   =====> Loss= 0.082501368 Accuracy: 0.9808
Epoch:  04   =====> Loss= 0.067433696 Accuracy: 0.9803
Epoch:  05   =====> Loss= 0.057805305 Accuracy: 0.9846
Epoch:  06   =====> Loss= 0.052207252 Accuracy: 0.9842
Epoch:  07   =====> Loss= 0.045424073 Accuracy: 0.985
Epoch:  08   =====> Loss= 0.042473145 Accuracy: 0.9854
Epo

<div class="alert alert-success">
**Evaluating the impact of the nb of epochs** <br>
As 40 epochs were not enough to reach a 99% accuracy, let's try with the following parameters : nb of epochs 100, optimizer : SDG
</div>

In [23]:
tf.reset_default_graph()
learning_rate = 0.001
training_epochs = 100
batch_size = 128
logs_path = 'SDG100_files/'
saving_path ='models/'
display_step = 10
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model(x)

# ----- Loss function -----
with tf.name_scope('Lossdrop'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), reduction_indices=1))
    
with tf.name_scope('Accuracydrop'):
    acc = evaluate(model,y)

with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
        
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_100epoch", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_100epoch", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  10   =====> Loss= 0.424216607 Accuracy: 0.8886
Epoch:  20   =====> Loss= 0.272833335 Accuracy: 0.9263
Epoch:  30   =====> Loss= 0.212675605 Accuracy: 0.9432
Epoch:  40   =====> Loss= 0.177034984 Accuracy: 0.9544
Epoch:  50   =====> Loss= 0.152907702 Accuracy: 0.9596
Epoch:  60   =====> Loss= 0.135107857 Accuracy: 0.9643
Epoch:  70   =====> Loss= 0.121158943 Accuracy: 0.9676
Epoch:  80   =====> Loss= 0.110109265 Accuracy: 0.9706
Epoch:  90   =====> Loss= 0.101161493 Accuracy: 0.9721
Ep


<img src="MNIST_figures/SDG100_Accuracy.JPG",width="800" height="600" align="center">
<center><span>Figure 3a: Accuracy, SDG, 100 epochs </span></center>


<img src="MNIST_figures/SDG100_Loss.JPG",width="800" height="600" align="center">
<center><span>Figure 3b: Loss, SDG, 100 epochs </span></center>

<div class="alert alert-success">
**Evaluating the impact of the nb of epochs** <br>
Increasing the number of epochs increases the accuracy from 0.95 to 0.973. 
Yet, that doesn't seem to be a great improvement as a better accuracy as achieved using Adam's optimizer with only 3 epochs. 
</div>

<div class="alert alert-success">

**Let's try and fix the stability issue of the Loss function when using the Adam optimizer**  <br>
This instability is due to the fact that the loss is so close to 0 the algorithm applies wrong optimizations. 
Then, we need to modify the Loss function by adding a small constant <br>
__cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model+0.001), reduction_indices=1))__

</div>

In [10]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 40
batch_size = 128
logs_path = 'AdamLoss_files/'
saving_path ='models/'
display_step = 1
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model(x)

# ----- Loss function -----
with tf.name_scope('LossAdam2'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model+0.001), reduction_indices=1))
    
with tf.name_scope('AccuracyAdam'):
    acc = evaluate(model,y)


with tf.name_scope('Adam'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_Adam", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_Adam", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  01   =====> Loss= 0.349506255 Accuracy: 0.9657
Epoch:  02   =====> Loss= 0.095894996 Accuracy: 0.9783
Epoch:  03   =====> Loss= 0.068343517 Accuracy: 0.9824
Epoch:  04   =====> Loss= 0.053774344 Accuracy: 0.9871
Epoch:  05   =====> Loss= 0.045449615 Accuracy: 0.9864
Epoch:  06   =====> Loss= 0.038054175 Accuracy: 0.9844
Epoch:  07   =====> Loss= 0.031517917 Accuracy: 0.9856
Epoch:  08   =====> Loss= 0.026478725 Accuracy: 0.9863
Epoch:  09   =====> Loss= 0.023710335 Accuracy: 0.9884
Ep

<img src="MNIST_figures/Adam_Accuracy.JPG",width="800" height="600" align="center">
<center><span>Figure 4a: Adam Accuracy </span></center>


<img src="MNIST_figures/Adam_Loss.JPG",width="800" height="600" align="center">
<center><span>Figure 4b: Adam Loss </span></center>

<div class="alert alert-success">
Then, with the loss function modified, with 40 epochs and the Adam's optimizer we reach an **accuracy of 0.9912** ! It is the best model so far in terms of accuracy and time performances.
</div>

<div class="alert alert-success">
**Let's now use the following parameters** <br>
optimizer : Adam <br>
epochs : 100 <br>
dropout layer : yes <br>
with the loss function modified
</div>

In [13]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 128
logs_path = 'Adam100_files/'
saving_path ='models/'
display_step = 10
# Model, loss function and accuracy

# ----- Define the model -----
x = tf.placeholder(tf.float32, [None, 784], name='InData')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model_Dropout(x)

# ----- Loss function -----
with tf.name_scope('LossAdam100'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model+0.001), reduction_indices=1))
    
with tf.name_scope('AccuracyAdam100'):
    acc = evaluate(model,y)


with tf.name_scope('Adam'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_Adam", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_Adam", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("dropout/mul:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  10   =====> Loss= 0.033881307 Accuracy: 0.9859
Epoch:  20   =====> Loss= 0.018523285 Accuracy: 0.9897
Epoch:  30   =====> Loss= 0.013141273 Accuracy: 0.9891
Epoch:  40   =====> Loss= 0.010596964 Accuracy: 0.9884
Epoch:  50   =====> Loss= 0.007855424 Accuracy: 0.9904
Epoch:  60   =====> Loss= 0.007567481 Accuracy: 0.9895
Epoch:  70   =====> Loss= 0.007358544 Accuracy: 0.991
Epoch:  80   =====> Loss= 0.006445506 Accuracy: 0.9906
Epo

<img src="MNIST_figures/Adam100_Accuracy.JPG",width="800" height="600" align="center">
<center><span>Figure 5a: Adam Accuracy </span></center>


<img src="MNIST_figures/Adam100_Loss.JPG",width="800" height="600" align="center">
<center><span>Figure 5b: Adam Loss </span></center>

<div class="alert alert-success">
Then, we obtained an Accuracy of 0.988 with the last model (optimizer = Adam, nb epoch = 100, Loss function modified). 
Again, the dropout layer does not present any improvement on the training set. 
</div>

In [11]:
tf.reset_default_graph()

learning_rate = 0.001
training_epochs = 100
batch_size = 128
logs_path = 'Adam100_files/'
saving_path ='models/'
display_step = 10

x = tf.placeholder(tf.float32, [None, 784], name='DataIn')
y = tf.placeholder(tf.float32, [None, 10], name='PrData')
model = LeNet5_Model(x)

# ----- Loss function -----
with tf.name_scope('LossAdam100nodrop'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(model+0.001), reduction_indices=1))
    
with tf.name_scope('AccuracyAdam100nodrop'):
    acc = evaluate(model,y)


with tf.name_scope('Adam'):
        # Gradient Descent
        optimizer = tf.train.AdamOptimizer(learning_rate,beta1=0.9,
    beta2=0.999,
    epsilon=1e-08,
    use_locking=False,
    name='Adam').minimize(cost)
init = tf.global_variables_initializer()
tf.summary.scalar("Loss_Adam", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy_Adam", acc)
merged_summary_op = tf.summary.merge_all()

train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 6), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 6), dtype=float32)
Tensor("Relu_1:0", shape=(?, 10, 10, 16), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 5, 5, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 400), dtype=float32)
Tensor("Relu_2:0", shape=(?, 120), dtype=float32)
Tensor("Relu_3:0", shape=(?, 84), dtype=float32)
Tensor("Softmax:0", shape=(?, 10), dtype=float32)
Starting Training
Epoch:  10   =====> Loss= 0.019452982 Accuracy: 0.9879
Epoch:  20   =====> Loss= 0.009337226 Accuracy: 0.9895
Epoch:  30   =====> Loss= 0.005278638 Accuracy: 0.9902
Epoch:  40   =====> Loss= 0.001642457 Accuracy: 0.9911
Epoch:  50   =====> Loss= 0.004150455 Accuracy: 0.9903
Epoch:  60   =====> Loss= 0.004520883 Accuracy: 0.9909
Epoch:  70   =====> Loss= 0.001729269 Accuracy: 0.9918
Epoch:  80   =====> Loss= 0.003938103 Accuracy: 0.9914
Epoch:  90   =====> Loss= 0.001266730 Accuracy: 0.9918
Ep

<div class="alert alert-success">
**Using the following parameters** <br>
optimizer : Adam <br>
epochs : 100 <br>
dropout layer : no <br>
with the loss function modified
**we reached an accuracy of 0.9924**
</div>